这个文件主要是根据法律的常识(而非仅仅依靠语言和逻辑的理解), 去补充MEU之间的关系 (and, or, not). 

In [ ]:
import os
import re
import csv
import json
from tqdm import tqdm
from utils.call_gpt import call_gpt, call_gpt_async

# 调用LLM获取MEU

## prompt 模板

In [2]:
prompt_meu_inner_relation_v1 = """
# MEU关系识别指令 inner relation "or"

## 角色定位
你是一个资深法律条款引用分析专家，专注识别同一个法条下若干 MEU (法律的最小可执行单元) 之间的 "or" 关系

## MEU概念简述
MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号. 第k个法条编号为Law_k. 
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 负责会判定该案例在该MEU上不违规. 


## 核心任务
从给定的MEU列表中识别 or 关系，输出(source_id, or, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, or, [target_id_1, target_id_2])
当所给的MEU之间或者MEU与法律之间不存在or时, 如实返回空值, 不需要自己杜撰或强行拼凑or. MEU间不存在关系是常见的现象. 


## or关系的含义与注意事项
- or关系的含义: MEU之间默认的关系是and, 也即只有当整个法条Law_k下全部MEU都不违规, 这个法条才会被判定为不违规. 但有时这种简单的结构无法涵盖法律的本意, 需要我们判断是否存在 or 的情况. 如果两个MEU被判定为 or 的关系, 那么只要这两个MEU之间有一个不违规, 这两个MEU便会一起被判定为不违规. 
- 例子:
法条原文: 
    "第五条 上市公司大股东、董监高应当在股份减持计划实施完毕或者披露的减持时间区间届满后，及时向本所报告并披露减持结果公告。..."
拆分得MEU:
    [{{
    "MEU_id": "MEU_5_1",
    "subject": "上市公司大股东 | 董监高",
    "condition": "股份减持计划实施完毕",
    "constrain": "应当及时向本所报告并披露减持结果公告",
    "contextual_info": "",
    }},{{
    "MEU_id": "MEU_5_2",
    "subject": "上市公司大股东 | 董监高",
    "condition": "披露的减持时间区间届满",
    "constrain": "应当及时向本所报告并披露减持结果公告",
    "contextual_info": "",
    }},...]
得到关系:
    思考: 虽然从语义和逻辑上看, 原文确实是在股份减持计划实施完毕和披露的减持时间区间届满这两种情况下, 都应当向本所报告并披露减持结果公告, 但考虑到业务逻辑, 单一的减持计划应当只需要披露一次减持结果公告. 因此, MEU_5_1和MEU_5_2不适用默认的"and"关系(否则减持计划实施完毕时披露了减持结果, 而减持时间区间届满没有再披露一次的案例会被误分类为违约), 而应当采用"or"的关系. 
    输出:
    <RELATIONS>
    [
    ("MEU_5_1", "or", "MEU_5_2"),
    ]
    </RELATIONS>


## 更多经验
- "or" 的关系是双向的, 因此对于("MEU_n_i", "or", "MEU_n_j")只需要输出一次即可, 无需再输出对向的("MEU_n_j", "or", "MEU_n_i")
- 并非法律原文中所有的"或者"都会构成MEU之间的or关系. 例如, condition中出现或的"当主体A在面临情况B或C时, 应当遵守约束D", 可以平行拆开为两个MEU[主体A面临情况B应当遵守D, 主体A面临情况C应当遵守D], 此时这两个MEU是默认的and关系. 但如果从业务的角度理解,  "遵守D"只需要发生一次, 主体A在面临B时进行D或者面临C时进行D, 只一次就好, 那么这两个拆开后的MEU就是or的关系. 
- 从业务的角度出发进行理解, 而不是从语言和逻辑的角度理解. 例如, "第十五条 上市公司收到上交所或深交所受理或者不受理、中止或者终止审核、同意转板申请相关文书后，应当及时予以披露。", 很明显这个法条要求在遇到受理, 不受理, 中止, 终止审核, 同意转板申请等相关文书后, 都要进行披露, 这个法条产生的MEU也就可以被默认的and关系表示, 不需要设置or的关系. 


## 识别原则
1. 不修改MEU内容，仅建立关联
2. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("MEU_n_i", "or", "MEU_n_j"),
("MEU_n_k", "or", "MEU_n_r"),
]
</RELATIONS>


# 接下来是作为参考的法律原文, 和等待你发掘关系的MEU列表:

## 法律原文
{law_article}

## MEU列表
{MEU_list}

"""

## 批量调用llm

In [ ]:
import csv
import json
import asyncio
from tqdm.asyncio import tqdm_asyncio
from utils.call_gpt import call_gpt_async
import os

async def process_law_articles_async(
    file_name,
    max_concurrency: int = 5,
    encoding: str = 'utf-8-sig'
):
    """
    异步处理法律条文的函数，支持并行控制
    
    参数：
    input_file: 输入文件路径
    output_file: 输出文件路径
    max_concurrency: 最大并行请求数 (默认5)
    encoding: 文件编码 (默认utf-8-sig)
    """

    if file_name.endswith(".csv"):
        pass
    else:
        file_name += ".csv"

    input_dir_law = r"law_to_MEU/st_1_law_csv"
    input_dir_MEU = r"law_to_MEU/st_3_0_MEU/GT"
    output_dir = r"law_to_MEU/st_3_1_inner_relations/raw_response"
    os.makedirs(output_dir, exist_ok=True)

    input_path_law = os.path.join(input_dir_law, file_name)
    input_path_MEU = os.path.join(input_dir_MEU, file_name)
    output_path = os.path.join(output_dir, file_name)

    # 读取CSV文件
    with open(input_path_law, mode='r', encoding=encoding) as infile:
        reader = csv.DictReader(infile)
        law_articles = list(reader)
    
    # 读取CSV文件
    with open(input_path_MEU, mode='r', encoding=encoding) as infile:
        reader = csv.DictReader(infile)
        MEU_list_all = list(reader)
    
    # 读取MEU数据并结构化分组
    meu_dict = {}
    for meu in MEU_list_all:
        # 解析MEU_id结构
        # 格式验证（MEU_n_k）
        prefix, law_num, meu_num = meu["MEU_id"].split("_")
        if prefix != "MEU" or not law_num.isdigit():
            continue

        # 初始化字典条目
        if law_num not in meu_dict:
            meu_dict[law_num] = []
        
        # 创建易读格式
        formatted_meu = (
            f"MEU ID: {meu['MEU_id']}\n"
            f"Subject: {meu['subject']}\n"
            f"Condition: {meu['condition']}\n"
            f"Constraint: {meu['constraint']}\n"
            f"Contextual Info: {meu['contextual_info']}"
        )
        meu_dict[law_num].append(formatted_meu)

    # 共享状态容器
    class State:
        def __init__(self):
            self.total_prompt_tokens = 0
            self.total_completion_tokens = 0
            self.success_count = 0
            self.failure_count = 0
            self.processed_data = []
            self.lock = asyncio.Lock()

    state = State()

    async def process_article(article):
        """处理单个法条的异步任务"""
        nonlocal state
        law_article_num = article['law_article_num']
        law_article = article['law_article']
        
        
        try:

            # 获取当前法条对应的MEU列表（空列表作为默认值）
            # 获取结构化MEU数据
            related_meu = meu_dict.get(law_article_num, [])
            # 转换为JSON字符串（保持缩进格式）
            MEU_list = json.dumps(related_meu, ensure_ascii=False, indent=2)

            prompt = prompt_meu_inner_relation_v1.format(
                law_article=law_article,
                MEU_list=MEU_list,
            )
            
            # 调用异步接口
            content, reasoning_content, api_usage = await call_gpt_async(
                prompt=prompt,
                api_key="35684824-1776-48b6-94fd-96c2e99d0724",
                base_url="https://ark.cn-beijing.volces.com/api/v3",
                model="ep-20250217153824-9xcbx",
                temperature=0.6,
            )
            
            # 原子操作更新状态
            async with state.lock:
                state.total_prompt_tokens += api_usage.prompt_tokens
                state.total_completion_tokens += api_usage.completion_tokens
                state.success_count += 1
                state.processed_data.append({
                    'law_article_num': law_article_num,
                    'law_article': law_article,
                    'response': content,
                    'reasoning_content': reasoning_content,
                    'api_usage': api_usage
                })
            
            return True
        except Exception as e:
            async with state.lock:
                state.failure_count += 1
                print(f"Failed to process law article {law_article_num}: {e}")
            return False

    # 创建异步任务
    tasks = [process_article(article) for article in law_articles]
    
    # 使用tqdm进度条分批执行
    pbar = tqdm_asyncio(total=len(tasks), desc="Processing law articles")
    
    # 分批执行控制并发
    for i in range(0, len(tasks), max_concurrency):
        batch_tasks = tasks[i:i + max_concurrency]
        await asyncio.gather(*batch_tasks)
        pbar.update(len(batch_tasks))
    
    pbar.close()

    # 保存结果到CSV
    with open(output_path, mode='w', encoding=encoding, newline='') as outfile:
        fieldnames = ['law_article_num', 'law_article', 'response', 'reasoning_content', 'api_usage']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for data in state.processed_data:
            writer.writerow(data)
    

In [4]:
# await process_law_articles_async("北京证券交易所上市公司持续监管指引第7号——转板.csv")

## 从LLM回复中提取MEU relation

In [8]:
import csv
import re
import json
import os

def get_relations_from_responses(file_name):
    if file_name.endswith(".csv"):
        pass
    else:
        file_name += ".csv"

    input_dir = r"law_to_MEU/st_3_1_inner_relations/raw_response"
    output_dir = r"law_to_MEU/st_3_1_inner_relations"
    os.makedirs(output_dir, exist_ok=True)

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)

    def clean_response(response):
        """清理和解析response列中的关系数据"""
        try:
            # 统一处理转义符号
            response = response.replace("<\\RELATIONS>", "</RELATIONS>")
            response = response.replace("<\\\\RELATIONS>", "</RELATIONS>")
            
            # 提取所有RELATIONS标签内容
            matches = re.findall(r'<RELATIONS>(.*?)</RELATIONS>', response, re.DOTALL)
            
            if matches:
                # 选择内容最长的匹配项
                longest_content = max(matches, key=lambda x: len(x.strip())).strip()
                
                # 数据格式转换
                longest_content = longest_content.replace('""', '"')  # 处理双引号转义
                longest_content = longest_content.replace('(', '[').replace(')', ']')  # 转换括号格式
                
                # 解析为JSON数组
                return json.loads(longest_content)
            return []
        except json.JSONDecodeError as e:
            print(f"JSON解析错误: {e}")
            print(f"问题内容: {longest_content}")
            return []
        except Exception as e:
            print(f"数据处理异常: {e}")
            return []

    # 读取原始CSV文件
    with open(input_path, mode='r', encoding='utf-8-sig') as infile:
        reader = csv.DictReader(infile)
        law_articles = list(reader)

    # 处理关系数据
    split_data = []
    fieldnames = ['source', 'relation', 'target']
    
    for row in law_articles:
        relations = clean_response(row['response'])
        for rel_entry in relations:
            # 校验数据格式
            if not isinstance(rel_entry, list) or len(rel_entry) != 3:
                print(f"异常条目: {rel_entry}")
                continue
            
            source, relation, targets = rel_entry
            # 处理target为列表的情况
            if isinstance(targets, list):
                for target in targets:
                    split_data.append({
                        'source': source.strip('"'),  # 去除可能的残留引号
                        'relation': relation.strip('"'),
                        'target': target.strip('"')
                    })
            else:
                split_data.append({
                    'source': source.strip('"'),
                    'relation': relation.strip('"'),
                    'target': targets.strip('"')
                })

    # 写入处理后的CSV文件
    with open(output_path, mode='w', encoding='utf-8-sig', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(split_data)
    
    print(f'文件已保存至: {output_path}')

In [6]:
# get_relations_from_responses("北京证券交易所上市公司持续监管指引第4号——股份回购.csv")

# 主workflow

In [4]:
import os
# 定义文件目录路径
directory_path = r"law_to_MEU/st_3_0_MEU/GT"

# 提取所有.doc文件的文件名，不包含路径
filenames = [
    f for f in os.listdir(directory_path) if f.endswith('.csv')
]


filenames

['北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv',
 '北京证券交易所上市公司持续监管指引第5号——要约收购.csv',
 '北京证券交易所上市公司持续监管指引第1号——独立董事.csv',
 '北京证券交易所上市公司持续监管指引第4号——股份回购.csv',
 '北京证券交易所上市公司持续监管指引第10号——权益分派.csv']

In [9]:
# for doc_file in doc_filenames:
#     main(doc_file)

for filename in filenames:
    # await process_law_articles_async(filename)
    get_relations_from_responses(filename)



文件已保存至: law_to_MEU/st_3_1_inner_relations/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv
文件已保存至: law_to_MEU/st_3_1_inner_relations/北京证券交易所上市公司持续监管指引第5号——要约收购.csv
文件已保存至: law_to_MEU/st_3_1_inner_relations/北京证券交易所上市公司持续监管指引第1号——独立董事.csv
文件已保存至: law_to_MEU/st_3_1_inner_relations/北京证券交易所上市公司持续监管指引第4号——股份回购.csv
文件已保存至: law_to_MEU/st_3_1_inner_relations/北京证券交易所上市公司持续监管指引第10号——权益分派.csv
